In [1]:
# Install Tableau Server Client (TSC)
!pip install tableauserverclient PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.0 MB/s eta 0:00:00


# **Login to Tableau Cloud Site**

In [3]:
import tableauserverclient as TSC
from google.colab import userdata


my_server_url = 'https://10ax.online.tableau.com/'
my_token_name = userdata.get('TC_token_name')
my_token_secret = userdata.get('TC_token_secret')
my_site_id = userdata.get('TC_site_id')
my_data_source = "cpat_results"


tableau_auth = TSC.PersonalAccessTokenAuth( token_name=my_token_name
                                           ,personal_access_token=my_token_secret
                                           ,site_id=my_site_id)
server = TSC.Server(my_server_url, use_server_version=True)

# Sign in to Tableau Cloud

server.auth.sign_in_with_personal_access_token(tableau_auth)
print('[Logged in successfully to {}]'.format(my_server_url))

[Logged in successfully to https://10ax.online.tableau.com/]


# **Retrieve only the requested Datasource and Populate Connection Details**

In [12]:
# Add filteration clause to consider only the requested workbook
req_option = TSC.RequestOptions()
req_option.filter.add(TSC.Filter(TSC.RequestOptions.Field.Name
                                ,TSC.RequestOptions.Operator.Equals
                                ,my_data_source))

# Print workbook details
datasources, pagination_item = server.datasources.get(req_options=req_option)
if datasources:
  ds = datasources[0]
  print(f"=== Datasource ===")
  print(f"Name:{ds.name}")
  print(f"luid:{ds.id}")
  print(f"URL:{ds.webpage_url}")
  print(f"Datasource:{ds.content_url}")
  print("==================")
  print("\n")

# get the connection information
  server.datasources.populate_connections(ds)
  connection = ds.connections[0]
  print(f"Datasource Connection Details:")
  print(f"Connection Type:\t{connection.connection_type}")
  print(f"Connection luid:\t{connection.id}")
  print(f"Server Address:\t{connection.server_address}")
  print(f"Server Port:\t{connection.server_port}")
  print(f"Username:\t{connection.username}")

=== Datasource ===
Name:cpat_results
luid:b35e6d3a-f6c5-4e3a-942b-829d35e68111
URL:https://10ax.online.tableau.com/#/site/setitsandboxdev427435/datasources/7577883
Datasource:cpat_results


=== Datasource Connection Details ===
Connection Type:	postgres
Connection luid:	a43a39e2-b652-4ff3-b0be-df22c850dd35
Server Address:	hh-pgsql-public.ebi.ac.uk
Server Port:	5432
Username:	readers


**The retrieved value of** `Username:	readers` **is not correct, Let's fix that**

In [13]:
# Assign the correct value for Username

connection.username = 'reader'

# **Update Connection Details**

In [16]:
# update datasource connection

server.datasources.update_connection(ds, connection)

<ConnectionItem#a43a39e2-b652-4ff3-b0be-df22c850dd35 embed=True type=None username=reader>

# **Let's Validate the updated connection**

In [17]:
# get the connection information

server.datasources.populate_connections(ds)
connection = ds.connections[0]
print(f"Datasource Connection Details:")
print(f"Connection Type:\t{connection.connection_type}")
print(f"Connection luid:\t{connection.id}")
print(f"Server Address:\t{connection.server_address}")
print(f"Server Port:\t{connection.server_port}")
print(f"Username:\t{connection.username}")

Datasource Connection Details:
Connection Type:	postgres
Connection luid:	a43a39e2-b652-4ff3-b0be-df22c850dd35
Server Address:	hh-pgsql-public.ebi.ac.uk
Server Port:	5432
Username:	reader


# **Sign out from Tableau Cloud**

In [ ]:
# Sign Out from Tableau Cloud

server.auth.sign_out()

# **Refrences:**

1. https://tableau.github.io/server-client-python/docs/api-ref#data-sources

